INTERMEDIATE MACHINE LEARNING


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Reading the training and test datasets
X_full = pd.read_csv('../input/train.csv', index_col = 'Id')
X_test_full = pd.read_csv('../input/test.csv', index_col = 'Id')

# Defining the target variable (SalePrice) and features for training
y = X_full.SalePrice
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = X_full[features].copy()
X_test= X_test_full[features].copy()

# Splitting the dataset into training and validation sets (80% training, 20% validation)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 0)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Defining different Random Forest models with varying hyperparameters
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='absulate_error', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)
models = [model_1, model_2, model_3, model_4, model_5]

In [ ]:
from sklearn.metrics import mean_absolute_error

# Function to evaluate a model using Mean Absolute Error (MAE)
def score_model(model, X_t=X_train, X_v=X_valid, y_t=y_train, y_v=y_valid):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

# Loop over the models starting from index 9 (10th model)
for i in range(9, len(models)):
  mae = score_model(models[i])
  print(models[i])
  print("Model %d MAE: %d" % (i+1, mae))


MISSING VALUES

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dansbecker/melbourne-housing-snapshot")

print("Path to dataset files:", path)


100%|██████████| 451k/451k [00:00<00:00, 846kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/dansbecker/melbourne-housing-snapshot/versions/5


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
#/content/melb_data.csv

# Read the dataset and separate target and features
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')
y = data.Price
melb_predictors = data.drop(['Price'], axis=1)
# Select only numerical features for training
X = melb_predictors.select_dtypes(exculde = ['object'])

# Split the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

FileNotFoundError: [Errno 2] No such file or directory: '../input/melbourne-housing-snapshot/melb_data.csv'

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
  model = RandomForestRegressor(n_estimators=10, random_state=0)
  model.fit(X_train, y_train)
  preds = model.predict(X_valid)
  return mean_absolute_error(y_valid, preds)

In [ ]:
# Approach 1: Drop columns with missing values
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

# Drop columns with missing values from the training and validation sets
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

In [ ]:
#imputation
from sklearn.impute import SimpleImputer

# Create an imputer object to fill missing values
my_imputer = SimpleImputer()
# Impute missing values in the training and validation datasets
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.Dataframe(my_imputer.transform(X_valid))

# Assign the original column names back to the imputed datasets
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

In [ ]:
#extention to imputation
# Approach 3: Imputation with a new feature indicating if a value was missing
X_train_plus = X_valid.copy()
X_valid_plus = X_valid.copy()

# For each column with missing values, add a new column that indicates if the value was missing
for col in cols_with_missing:
  X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
  X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Impute the missing values again in the training and validation datasets
  my_imputer = SimpleImputer()
  imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
  imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Assign the original column names back to the imputed datasets
  imputed_X_train_plus.columns = X_train_plus.columns
  imputed_X_valid_plus.columns = X_valid_plus.columns

  print("MAE from Approach 3 (An Extension to Imputation):")
  print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))
#

In [ ]:
print(X_train.shape)

missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
#categorial variables
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

# Define target and predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Split the dataset into training and validation sets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Identify columns with missing values
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()]

# Drop columns with missing values
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# Identify low cardinality categorical columns (columns with fewer than 10 unique values)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == 'object']

# Identify numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Combine low cardinality columns and numerical columns for feature selection
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

# Identify categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

# Print the list of categorical variables
print("Categorical variables:")
print(object_cols)

# Importing RandomForestRegressor and mean_absolute_error for model evaluation
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function to evaluate a model with MAE
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Approach 1: Drop categorical variables
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

# Print MAE for the approach where categorical variables are dropped
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function to evaluate a model with MAE
def score_dataset(X_train, X_valid, y_train, y_valid):
  model = RandomForestRegressor(n_estimators=100, random_state=0)
  model.fit(X_train, y_train)
  preds = model.predict(X_valid)
  return mean_absolute_error(y_valid, preds)


In [ ]:
#drop categorial variables
# Approach 1: Drop categorical variables
drop_X_train = X_train.select.dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))


In [ ]:
#ordinal encoding
# Approach 2: Ordinal Encoding for categorical variables
from sklearn.preprocessing import OrdinalEncoder

# Copy the data for encoding
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply Ordinal Encoding to categorical columns
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))


In [ ]:
# Approach 3: One-Hot Encoding for categorical variables
from sklearn.preprocessing import OneHotEncoder

# Apply One-Hot Encoding to categorical columns
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# Set the index of the new encoded columns to match the original data
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Drop categorical columns and concatenate the One-Hot Encoded columns
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Convert column names to strings for consistency
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

# Print MAE for the approach using One-Hot Encoding
print("MAE from Approach 3 (One-Hot Encoding):")
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

PIPELINES

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the dataset again
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

# Define target and predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Split the data into training and validation sets
X_train_full, X_valid_full, y_train_full, y_valid_full = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Identify categorical and numerical columns
categorial_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == 'object']
numeriacal_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Select columns for feature set
my_cols = categorial_cols + numeriacal_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Create transformer for numerical data (constant imputation)
numerical_transformer = SimpleImputer(strategy='constant')

# Create transformer for categorical data (most frequent imputation and One-Hot Encoding)
categorial_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine both transformers into a preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols), ('cat', categorial_transformer, categorial_cols)])


In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)


In [ ]:
from sklearn.metrics import mean_absolute_error

# Create a pipeline that first processes the data and then fits the model
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Fit the pipeline to the training data
my_pipeline.fit(X_train, y_train)

# Make predictions on the validation data
preds = my_pipeline.predict(X_valid)

# Calculate and print the MAE for the pipeline model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)